Trong hướng dẫn này, bạn sẽ học cách sử dụng **pipelines** để tổ chức mã mô hình của mình.

# Giới thiệu (# Introduction #)

**Pipelines** là một cách đơn giản để giữ cho mã tiền xử lý dữ liệu và mô hình hóa được tổ chức gọn gàng. Cụ thể, một *pipeline* gói gọn các bước tiền xử lý và mô hình hóa, giúp bạn có thể sử dụng toàn bộ quy trình này như một bước duy nhất.

Nhiều *data scientists* phát triển mô hình mà không sử dụng *pipelines*, nhưng *pipelines* có một số lợi ích quan trọng, bao gồm:

1. **Mã gọn gàng hơn**: Việc xử lý dữ liệu ở từng bước tiền xử lý có thể trở nên phức tạp. Với một *pipeline*, bạn không cần phải theo dõi thủ công dữ liệu huấn luyện và kiểm định tại mỗi bước.  
2. **Giảm lỗi**: Ít có khả năng áp dụng sai hoặc quên một bước tiền xử lý.  
3. **Dễ dàng triển khai vào sản phẩm**: Chuyển một mô hình từ giai đoạn thử nghiệm sang triển khai quy mô lớn có thể rất khó khăn. Chúng ta sẽ không đi sâu vào các vấn đề liên quan ở đây, nhưng *pipelines* có thể giúp ích đáng kể.  
4. **Thêm nhiều tùy chọn để đánh giá mô hình**: Bạn sẽ thấy một ví dụ trong hướng dẫn tiếp theo, nơi chúng ta sẽ tìm hiểu về *cross-validation* (*xác thực chéo*).  

# Ví dụ (# Example #)

Giống như trong hướng dẫn trước, chúng ta sẽ làm việc với tập dữ liệu [Melbourne Housing](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot/home).  

Chúng ta sẽ không tập trung vào bước tải dữ liệu. Thay vào đó, giả sử rằng chúng ta đã có sẵn dữ liệu huấn luyện và kiểm định trong các biến:  
- `X_train`, `X_valid` (tập đặc trưng huấn luyện và kiểm định)  
- `y_train`, `y_valid` (tập mục tiêu huấn luyện và kiểm định)

In [6]:

import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

Chúng ta xem qua dữ liệu huấn luyện bằng phương thức `head()` dưới đây.  

Lưu ý rằng dữ liệu chứa cả biến phân loại (*categorical data*) và các cột có giá trị bị thiếu (*missing values*).  

Với *pipeline*, việc xử lý cả hai vấn đề này trở nên dễ dàng hơn!

In [7]:
X_train.head()

Type Method             Regionname  Rooms  Distance  Postcode  Bedroom2  \
12167    u      S  Southern Metropolitan      1       5.0    3182.0       1.0   
6524     h     SA   Western Metropolitan      2       8.0    3016.0       2.0   
8413     h      S   Western Metropolitan      3      12.6    3020.0       3.0   
2919     u     SP  Northern Metropolitan      3      13.0    3046.0       3.0   
6043     h      S   Western Metropolitan      3      13.3    3020.0       3.0   

       Bathroom  Car  Landsize  BuildingArea  YearBuilt  Lattitude  \
12167       1.0  1.0       0.0           NaN     1940.0  -37.85984   
6524        2.0  1.0     193.0           NaN        NaN  -37.85800   
8413        1.0  1.0     555.0           NaN        NaN  -37.79880   
2919        1.0  1.0     265.0           NaN     1995.0  -37.70830   
6043        1.0  2.0     673.0         673.0     1970.0  -37.76230   

       Longtitude  Propertycount  
12167    144.9867        13240.0  
6524     144.9005         6380.0  
8413     144.8220         3755.0  
2919     144.9158         8870.0  
6043     144.8272         4217.0

Chúng ta xây dựng *pipeline* hoàn chỉnh trong ba bước.

### Bước 1: Xác định các bước tiền xử lý (# Step 1: Define Preprocessing Steps #)

Tương tự như cách *pipeline* gói gọn các bước tiền xử lý và mô hình hóa, chúng ta sử dụng lớp `ColumnTransformer` để gộp các bước tiền xử lý khác nhau vào một quy trình duy nhất.  

Đoạn mã dưới đây thực hiện:
- Điền giá trị bị thiếu (*imputation*) cho dữ liệu **_số học_** (*numerical data*), và  
- Điền giá trị bị thiếu đồng thời áp dụng *one-hot encoding* cho dữ liệu **_phân loại_** (*categorical data*).  

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

### Bước 2: Xác định mô hình (# Step 2: Define the Model #)

Tiếp theo, chúng ta xác định một mô hình *random forest* bằng cách sử dụng lớp [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) quen thuộc.

In [9]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

### Bước 3: Tạo và đánh giá *Pipeline* (# Step 3: Create and Evaluate the Pipeline #)

Cuối cùng, chúng ta sử dụng lớp [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) để định nghĩa một *pipeline* gói gọn cả bước tiền xử lý và mô hình hóa.  

Có một số điểm quan trọng cần lưu ý:
- Với *pipeline*, chúng ta có thể tiền xử lý dữ liệu huấn luyện và huấn luyện mô hình chỉ trong một dòng lệnh.  
  (_Ngược lại, nếu không sử dụng *pipeline*, chúng ta phải thực hiện riêng lẻ các bước điền giá trị bị thiếu (*imputation*), mã hóa *one-hot encoding*, và huấn luyện mô hình. Điều này trở nên rất rối nếu phải xử lý cả biến số học và biến phân loại!_)  
- Với *pipeline*, chúng ta có thể cung cấp các đặc trưng thô (*unprocessed features*) trong `X_valid` vào lệnh `predict()`, và *pipeline* sẽ tự động tiền xử lý dữ liệu trước khi tạo dự đoán.  
  (_Tuy nhiên, nếu không sử dụng *pipeline*, chúng ta phải nhớ tiền xử lý dữ liệu kiểm định trước khi thực hiện dự đoán._)  

In [10]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 160679.18917034855


# Kết luận (# Conclusion #)

*Pipelines* rất hữu ích để làm gọn mã *machine learning* và giảm thiểu lỗi, đặc biệt quan trọng khi làm việc với quy trình tiền xử lý dữ liệu phức tạp.

# Đến lượt bạn! (# Your Turn #)

Hãy sử dụng *pipeline* trong [**bài tập tiếp theo**](https://www.kaggle.com/kernels/fork/3370278) để áp dụng các kỹ thuật tiền xử lý dữ liệu nâng cao và cải thiện độ chính xác dự đoán của bạn!

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intermediate-machine-learning/discussion) to chat with other learners.*